In [16]:
import os

input_dir = "../input/crack-seg/crack_segmentation_dataset/images"
target_dir = "../input/crack-seg/crack_segmentation_dataset/masks"
img_size = (448, 448)
num_classes = 1
batch_size = 8

input_img_paths = sorted(
    [
        os.path.join(input_dir, fname)
        for fname in os.listdir(input_dir)
        if fname.endswith(".jpg")
    ]
)
target_img_paths = sorted(
    [
        os.path.join(target_dir, fname)
        for fname in os.listdir(target_dir)
        if fname.endswith(".jpg") and not fname.startswith(".")
    ]
)

print("Number of samples:", len(input_img_paths))

for input_path, target_path in zip(input_img_paths[:10], target_img_paths[:10]):
    print(input_path, "|", target_path)
    print('oi')

Number of samples: 11298
../input/crack-seg/crack_segmentation_dataset/images/CFD_001.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_001.jpg
oi
../input/crack-seg/crack_segmentation_dataset/images/CFD_002.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_002.jpg
oi
../input/crack-seg/crack_segmentation_dataset/images/CFD_003.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_003.jpg
oi
../input/crack-seg/crack_segmentation_dataset/images/CFD_004.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_004.jpg
oi
../input/crack-seg/crack_segmentation_dataset/images/CFD_005.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_005.jpg
oi
../input/crack-seg/crack_segmentation_dataset/images/CFD_006.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_006.jpg
oi
../input/crack-seg/crack_segmentation_dataset/images/CFD_007.jpg | ../input/crack-seg/crack_segmentation_dataset/masks/CFD_007.jpg
oi
../input/crack-seg/crack_segmentation

In [7]:
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from PIL import Image

class Cracks(keras.utils.Sequence):
    """Helper to iterate over the data (as Numpy arrays)."""

    def __init__(self, batch_size, img_size, input_img_paths, target_img_paths):
        self.batch_size = batch_size
        self.img_size = img_size
        self.input_img_paths = input_img_paths
        self.target_img_paths = target_img_paths

    def __len__(self):
        return len(self.target_img_paths) // self.batch_size

    def __getitem__(self, idx):
        """Returns tuple (input, target) correspond to batch #idx."""
        i = idx * self.batch_size
        batch_input_img_paths = self.input_img_paths[i : i + self.batch_size]
        batch_target_img_paths = self.target_img_paths[i : i + self.batch_size]
        x = np.zeros((batch_size,) + self.img_size + (3,), dtype="float32")
        for j, path in enumerate(batch_input_img_paths):
            img = load_img(path, target_size=self.img_size)
            img_arr = np.asarray(img)
            img_arr = img_arr.astype(np.float32)
            img_arr = img_arr/255.0
            x[j] = img_arr
        y = np.zeros((batch_size,) + self.img_size + (1,), dtype="uint8")
        for j, path in enumerate(batch_target_img_paths):
            img = load_img(path, target_size=self.img_size, color_mode="grayscale")
            y[j] = np.expand_dims(img, 2)/255

        return x, y

In [17]:
import random

# Split our img paths into a training and a validation set
val_samples = 1000
random.Random(42).shuffle(input_img_paths)
random.Random(42).shuffle(target_img_paths)
train_input_img_paths = input_img_paths[:-val_samples]
train_target_img_paths = target_img_paths[:-val_samples]
val_input_img_paths = input_img_paths[-val_samples:]
val_target_img_paths = target_img_paths[-val_samples:]

# Instantiate data Sequences for each split
train_gen = Cracks(
    batch_size, img_size, train_input_img_paths, train_target_img_paths
)
val_gen = Cracks(batch_size, img_size, val_input_img_paths, val_target_img_paths)

In [18]:
import tensorflow as tf
model = tf.keras.models.load_model('../input/model-2/cracks_segmentation_v2.h5')

In [19]:
import pickle

val_gen = Cracks(batch_size, img_size, val_input_img_paths, val_target_img_paths)

val_preds = model.predict(val_gen)

In [20]:
val_preds_norm = preds*255

In [21]:
val_preds_norm_thres = (val_preds_norm > 51).astype(np.uint8)

In [22]:
def dice(y_true, y_pred):
    return (2 * (y_true * y_pred).sum() + 1e-15) / (y_true.sum() + y_pred.sum() + 1e-15)

ix = 0
dice_avg = 0
for batch in val_gen:
    for image in batch[1]:
        y_true = (image > 0).astype(np.uint8)
        y_pred = val_preds_norm_thres[ix]
        dice_avg += dice(y_true, y_pred)
        ix += 1

In [24]:
print("Avg. Dice:", dice_avg/1000)

Avg. Dice: 0.5765572559091258
